# Convertir el excel en dataframe 

In [1]:
# Obtener de los dos archivos excel 
# y juntar las bases de datos en un mismo dataframe
import pandas as pd 
df1 = pd.read_excel('210_SIVIGILA Escritorio_01042008-30042023.xls')
df2 = pd.read_excel('210_SIVIGILAweb_01052023-06082024 (1).xls')
df = pd.concat([df1, df2], ignore_index=True)
df.shape

(5300, 121)

In [2]:
# 2. Ordenar los datos de acuerdo a los atributos de año y semana epidemiológica
df = df.sort_values(by=['año', 'semana'])
df.shape

(5300, 121)

In [3]:
# Detección y eliminación de caracteres no implimibles en el dataset
import re

# Expresión regular para encontrar caracteres no imprimibles
illegal_char_re = re.compile(r'[\x00-\x1F\x7F-\x9F]')

def find_illegal_chars(df):
    for col in df.columns:
        for idx, val in df[col].items():
            if isinstance(val, str) and illegal_char_re.search(val):
                print(f"Carácter ilegal encontrado en fila {idx}, columna '{col}': {repr(val)}")


# corregir aquí 
def remove_illegal_chars(val):
    if isinstance(val, str):
        return illegal_char_re.sub('', val)
    return val

df = df.map(remove_illegal_chars)
df.shape 



(5300, 121)

In [4]:
# contando los diferentes barrios y veredas que hay en el dataset original 
df.bar_ver_.value_counts()

bar_ver_
9999999 SIN INFORMAC    203
PUEBLO NUEVO            181
ASOVIVIENDA             119
SIN DATO                 59
EL TRIANGULO             49
                       ... 
ALTOS DEL COUNTRY         1
BARRIO PORVENIR           1
BARRIO COSTA DE ORO       1
HACIENDA LA URIBE         1
BR PAJONAL                1
Name: count, Length: 463, dtype: int64

In [5]:
lista_barrios_correctos = ['PUEBLO NUEVO', 'CENTENARIO', \
                          'LAS VILLAS', 'EL TRIÁNGULO', 'EL BOSQUE', \
                          'PUEBLO SANTO', 'LOS ALMENDROS', 'EL DROMEDARIO', \
                          'EL CENTRO', 'VILLA ARABIA', 'CARACOLÍ', \
                          'VILLA GRANDA', 'ASOVIVIENDA', 'EL KENNEDY', \
                          'CLEMENTE ARRIETA', 'EL ÁGUILA', 'EL POBLADO', \
                          'SANTA ELENA', 'EL CAMELLO', 'LA ESPERANZA', \
                          'EL PRADO', 'SAN MIGUEL', 'SAN RAFAEL', \
                          'LA PAZ', 'LAS MALVINAS', 'EL CARMEN', \
                          'NUEVA ESTRELLA', 'LOMA FRESCA', 'EL ROBLE', \
                          'LA COLOMBIANITA', 'PARAGUAY', 'LA YE', \
                          'ALTOS DE SAN JUAN', 'PUERTO ESPAÑA', 'EL PALMAR', \
                          'LAS GAVIOTAS', 'ALTOS DE KIRIQUI', 'CASTILLITO', \
                          'LOS CÓRDOBA', 'EL LAGO', 'BUENO AIRES', \
                          'PEDRO VALDIVIA', 'SIN INFORMACION']


In [6]:
len(lista_barrios_correctos) 

43

In [ ]:
# cuantos datos nulos hay en ell atributo bar_ver_
df['bar_ver_'].isnull().sum() 

In [7]:
# llene los datos faltantes con la cadena SIN INFORMACION
df['bar_ver_'] = df['bar_ver_'].fillna('SIN INFORMACION') 

In [8]:
df['bar_ver_'].isnull().sum() 

np.int64(0)

In [34]:
import pandas as pd
from difflib import get_close_matches

# Lista de barrios correctos
lista_barrios_correctos = ['PUEBLO NUEVO', 'CENTENARIO', 'LAS VILLAS', 'EL TRIÁNGULO', 'EL BOSQUE', 
                           'PUEBLO SANTO', 'LOS ALMENDROS', 'EL DROMEDARIO', 'EL CENTRO', 
                           'VILLA ARABIA', 'CARACOLÍ', 'VILLA GRANDA', 'ASOVIVIENDA', 
                           'EL KENNEDY', 'CLEMENTE ARRIETA', 'EL ÁGUILA', 'EL POBLADO', 
                           'SANTA ELENA', 'EL CAMELLO', 'LA ESPERANZA', 'EL PRADO', 
                           'SAN MIGUEL', 'SAN RAFAEL', 'LA PAZ', 'LAS MALVINAS', 
                           'EL CARMEN', 'NUEVA ESTRELLA', 'LOMA FRESCA', 'EL ROBLE', 
                           'LA COLOMBIANITA', 'PARAGUAY', 'LA YE', 'ALTOS DE SAN JUAN', 
                           'PUERTO ESPAÑA', 'EL PALMAR', 'LAS GAVIOTAS', 'ALTOS DE KIRIQUI', 
                           'CASTILLITO', 'LOS CÓRDOBA', 'EL LAGO', 'BUENO AIRES', 
                           'PEDRO VALDIVIA', 'SIN INFORMACION']

def corregir_barrios(nombre):
    if pd.isnull(nombre):
        return 'SIN INFORMACION'
    
    # Convertir a cadena de texto
    nombre = str(nombre).strip().upper()
    
    coincidencias = get_close_matches(nombre, lista_barrios_correctos, n=1, cutoff=0.8)
    if coincidencias:
        return coincidencias[0]
    else:
        return 'SIN INFORMACION'

# Aplicar la corrección a la columna 'bar_ver'
df['bar_ver_corregido'] = df['bar_ver_'].apply(corregir_barrios)

In [35]:
'bar_ver_' in df.columns 

True

In [36]:
df['bar_ver_'].isnull().sum()

np.int64(0)

In [37]:
# verificación de la corrección del atributo bar_ver_  

len(df.bar_ver_corregido.value_counts())

40

In [41]:
df.bar_ver_.value_counts()[:40]

bar_ver_
SIN INFORMACION            3109
9999999 SIN INFORMAC        200
PUEBLO NUEVO                179
ASOVIVIENDA                 119
SIN DATO                     53
EL TRIANGULO                 49
VILLA ARABIA                 41
LAS MALVINAS                 41
SAN MIGUEL                   39
9999999 SIN INFORMACION      37
EL CAMELLO                   37
PARAGUAY                     36
EL CAMELLO 2                 35
SANTA ELENA                  34
EL PALMAR                    28
LA PLAYA                     27
LA COLOMBIANITA              24
CLEMENTE ARRIETA             24
EL POBLADO                   24
SAN RAFAEL                   22
BUENOS AIRES                 22
LA ESPERANZA                 19
EL CASTILLO                  19
CARACOLI                     18
LOMA FRESCA                  17
EL BOSQUE                    17
CAUCASIA                     17
EL AGUILA                    16
EL ROBLE                     16
LAS GAVIOTAS                 16
EL PRADO                     15

In [42]:
lista_atributos_criterio_igualdad = ['fec_not', 'semana', 'año','edad_', \
                                'nombre_nacionalidad', 'sexo_',\
                                'localidad_', 'vereda_', 'bar_ver_corregido', 'dir_res_',\
                                      'fec_con_', 'ini_sin_', 'fec_hos_',\
                                          'telefono_', 'fecha_nto_']
df = df.drop_duplicates(subset=lista_atributos_criterio_igualdad)
df.shape 

(5264, 122)

In [43]:
df = df.drop_duplicates(subset=['fec_not', 'semana', 'año','edad_', \
                                'nombre_nacionalidad', 'sexo_',\
                                'localidad_','bar_ver_corregido',\
                                      'fec_con_', 'ini_sin_', 'fec_hos_',\
                                          'telefono_', 'fecha_nto_'])
df.shape 

(5264, 122)

In [44]:
df.bar_ver_.value_counts()[:20]

bar_ver_
SIN INFORMACION            3109
9999999 SIN INFORMAC        200
PUEBLO NUEVO                179
ASOVIVIENDA                 119
SIN DATO                     53
EL TRIANGULO                 49
VILLA ARABIA                 41
LAS MALVINAS                 41
SAN MIGUEL                   39
9999999 SIN INFORMACION      37
EL CAMELLO                   37
PARAGUAY                     36
EL CAMELLO 2                 35
SANTA ELENA                  34
EL PALMAR                    28
LA PLAYA                     27
LA COLOMBIANITA              24
CLEMENTE ARRIETA             24
EL POBLADO                   24
SAN RAFAEL                   22
Name: count, dtype: int64

In [45]:
df.to_excel('datos_secretaria_con_casos_repetidos_eliminados.xlsx', index=False) 